# Supervisory Control Theory package

In [ ]:
from graphviz import Digraph
import pandas as pd

In [497]:
class SupBuilder(object):
    
    def __init__(self):
        self.__states = {}          # Dictionary containing states and transitions directions into a DataFrame
        self.__alphabet = set()     # Alphabet set
        self.__t_count = {}

    #----- STATES methods:

    '''
        Get the nodes that belong to the Supervisor.
    '''
    def show_states(self):
        for s in self.__states:
            print("From ", s, "to:\n")
            print(self.__states[s],"\n\n")
        print("\nAlphabet = ",self.__alphabet)
    
    '''
        Insert new node into the Supervisor automaton:
            node_name: UPPER_CASE name
    '''
    def state_insert(self, node_name):
        if not node_name.isupper():
            print("Incorrect node name. It must be upper_case!")
        else:
            if node_name not in self.__states:
                self.__states[node_name] = pd.DataFrame(columns=['transition', 'output_node'])   #Create a data frame for the new state
            else:
                print("There is already a node called \'",node_name,"\'")

    '''
        Remove a node from the Supervisor automaton:
    '''
    def state_remove(self, node_name):
        if node_name in self.__states:
            # Discount the counter of each transition starting on this node
            for t in self.__states[node_name]['transition']:
                self.__t_count[t] -= 1
                if self.__t_count[t]==0:
                    self.__alphabet.remove(t)      # Remove transition from alphabet if not used
            
            self.__states.pop(node_name)           # Remove the node
            
            #Remove each transition that ends on this node
            for s in self.__states:
                    # self.__states[s].drop(self.__states[s].loc[self.__states[s]['output_node'] == node_name].index, inplace=True)  #Remove edges to node
                    index = self.__states[s][self.__states[s]['output_node'] == node_name].index
                    self.__states[s].drop(index, inplace=True)  #Remove edges to node
                    
                    # self.__t_count[t] -= 1
                    # if self.__t_count[t]==0:
                    #     self.__alphabet.remove(t)      # Remove transition from alphabet if not used

        elif len(self.__states) == 0:
            print("Error: Empty supervisor!")
        else:
            print("Error: Absent node!")


    #----- TRANSITION methods:

    '''
        Insert a new transition:
            t_name: transistion name;
            in_node: start node of the transition;
            out_node: destiny node from transition.
    '''
    def insert_transition(self, t_name, in_node, out_node):
        # verify names 
        if t_name.islower() and out_node.isupper():   
            # verify in_node existence      
            if in_node in self.__states:
                # verify existence of the new transition
                if self.__states[in_node].loc[self.__states[in_node]['transition'] == t_name].empty:
                    self.__states[in_node] = self.__states[in_node].append({'transition': t_name , "output_node": out_node}, ignore_index=True)
                    if t_name not in self.__alphabet:
                        self.__t_count[t_name] = 1
                    else:
                        self.__t_count[t_name] += 1
                    self.__alphabet.add(t_name)
                else:
                    print("Transition \'",t_name,"\' already exist from state \'",in_node,"\'")
            else:
                print("Error: Absent node!")
        else:
            print("Incorrect names.\n\t Transition --> lower_case!\n\tState --> upper_case!")

    '''
        Remove a transition from in_node:
    '''
    def remove_transition(self, t_name, in_node):
        # verify in_node existence 
        if in_node in self.__states:
            # verify transition existence 
            if t_name in self.__alphabet:
                self.__states[in_node].drop(self.__states[in_node][self.__states[in_node]['transition'] == t_name].index,  inplace=True)  #Remove the desired transition 
                self.__t_count[t_name] -= 1
                if self.__t_count[t_name]==0:
                    self.__alphabet.remove(t_name)      # remove transition from alphabet if not used
            else:
                print("Error: Absent transition!")
        else:
            print("Error: Absent node!")


In [494]:
class SupMonitor(object):

    def __init__(self, supervisor):
        if not type(supervisor) == SupBuilder:
            print("Error!")
        else:
            self.__supervisor = supervisor


In [495]:
sup1 = SupBuilder()
sup1.state_insert('A')
sup1.state_insert('B')
sup1.state_insert('C')
sup1.insert_transition('t1','A','B')
sup1.insert_transition('t2','C','B')
sup1.insert_transition('t3','B','A')
sup1.show_states()

From  A to:

  transition output_node
0         t1           B 


From  B to:

  transition output_node
0         t3           A 


From  C to:

  transition output_node
0         t2           B 



Alphabet =  {'t3', 't1', 't2'}


In [496]:
sup1.state_remove('A')
sup1.show_states()

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [index]"

In [402]:
sup1.remove_transition('t1','B')